# VERIFICANDO SE TEM CUDA

In [ ]:
import torch
import torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA disponível:", torch.cuda.is_available())

# DATASET >> https://universe.roboflow.com/force-iaope/cars-egrgd/dataset/1#

# PRÉ-AMBULO - IMPORT LIBS

In [1]:
from ultralytics import YOLO
from PIL import Image
import cv2
import os
import imutils

from modules.helper.helper import create_video_from_images, get_framerate

# https://docs.ultralytics.com/datasets/

# TRAIN

In [ ]:
# # Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs # c:\dev\datasets
results = model.train(
    data="coco128.yaml", 
    epochs=1000,                # Number of epochs to train
    device="0", 
    workers=2,                  # Number of workers for data loading
    classes=[0, 1, 2, 3, 5, 7], # Alterar para as classes desejadas
    imgsz=1024,                 # Image size for training (960x960) 
    batch=4,                    # Batch size
    cache=True,                 # Cache images for faster training
    save_period=5,              # Save model every 5 epochs
    name="yolo11n_960_1000",    # Alterar para o nome desejado 
    exist_ok=True,              # Overwrite existing model directory
    project="runs/train",       # Alterar para o diretório desejado
    resume=True,                # Continue training from the last checkpoint
    patience=10,                # Early stopping patience
    optimizer="AdamW",          # Alterar para 'Adam' ou 'AdamW' se necessário
    pretrained=True,            # Use True para carregar pesos pré-treinados
    # hyp="data/hyp.scratch-low.yaml",
)

# Validate the model on the validation set
valid = model.val()

# Export the model to ONNX format
# success = model.export(format="onnx")

# Save the trained model for future reuse
model.save("trained_model/last.pt")

In [12]:
model.export(format="onnx", dynamic=True)

Ultralytics 8.3.99  Python-3.12.7 torch-2.6.0+cu118 CPU (12th Gen Intel Core(TM) i7-1255U)

PyTorch: starting from 'c:\dev\Contagem-Volumetrica\runs\detect\train21\weights\best.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 84, 18900) (5.3 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.49...
ONNX: export success  12.2s, saved as 'c:\dev\Contagem-Volumetrica\runs\detect\train21\weights\best.onnx' (10.1 MB)

Export complete (12.7s)
Results saved to C:\dev\Contagem-Volumetrica\runs\detect\train21\weights
Predict:         yolo predict task=detect model=c:\dev\Contagem-Volumetrica\runs\detect\train21\weights\best.onnx imgsz=960  
Validate:        yolo val task=detect model=c:\dev\Contagem-Volumetrica\runs\detect\train21\weights\best.onnx imgsz=960 data=C:\dev\Contagem-Volumetrica\venv\Lib\site-packages\ultralytics\cfg\datasets\coco128.yaml  
Visualize:       https://netron.app


'c:\\dev\\Contagem-Volumetrica\\runs\\detect\\train21\\weights\\best.onnx'

In [ ]:
# Save the trained model for future reuse
model.save("trained_model.pt")

# TEST IN VIDEO 12s

In [ ]:
# Caminho para o vídeo
video_path  = "input\\Teste.mp4"

save        = "output\\detect"
os.makedirs(save, exist_ok=True)

# Perform object detection on an image using the model
results = model(source=video_path)

# Visualize the results
for i, r in enumerate(results):
    try:
        # Plot results image
        r = imutils.resize(r, width=700)
        im_bgr = r.plot()  # BGR-order numpy array
        if im_bgr is None:
            print(f"Resultado {i} não contém dados válidos para plotar.")
            continue
        im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
        # Save results to disk
        r.save(filename=os.path.join(save, f"results{i}.jpg"))
    except ValueError as e:
        print(f"Erro ao processar o resultado {i}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao processar o resultado {i}: {e}")

# Exemplo de uso
image_folder = save
output_video_path = "output_video.mp4"
frame_rate = get_framerate(video_path)  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")